## Kindergarten Oral Testing GUI
There are typically 20-25 students in a Kindergarten class. Say, every month they have an oral exam for teachers to get an understanding of the progress of the students on how much they have learnt in that month. The teacher will typically call each student near her, ask the questions, write down the answers and their score. She will do this for each student, so on an average say 20 times. Can you imagine how much time and energy is spent unnecessarily by repeating the questions and recording everything by hand? What if all this could be done automatically?
Keeping these problems in mind, I was determined to find a solution easier for the teachers as well as interesting for the students. In today's age of internet and computers, we can find innovative solutions to minimize efforts and conveniently  infer student learning trends from the data.

This kindergarten oral testing GUI has the following features:
1. Take user input of student name and roll number.
2. Begin the test, record student answers for each question and calculate the score.
3. Restart the process for the next student.
4. Display a report in table format with name, roll number, score and answers for all the students. This report can be downloaded and further analyzed as well.
5. Create a database with the report table for future usage and analysis at the end of the year.
6. Generate and display graphs for count of students for each score, percentage of students for each score, the class statistics and list of students and their scores who need re-teaching.

I have used the standard GUI toolkit of tkinter for generating and displaying the testing GUI, pygame library for audio messages and deep learning speech_recognition to check student answers. I have given a sample of five questions in the testing GUI. These questions can easily be changed by the teacher by modifying the images, the questions and answers.

In [ ]:
#Import required libraries
from gtts import gTTS #Library and CLI tool to interface with Google Translate text-to-speech API
import speech_recognition as sr #Library for performing speech recognition
import os
import tkinter as tk #Standard GUI (Graphical User Interface) toolkit
from tkinter import messagebox
from pygame import mixer #Library for playing audio
from IPython.display import Image #Library to display image
from PIL import Image, ImageTk
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from pandastable import Table, TableModel #library with a table widget for Tkinter with plotting and data manipulation functionality

class Oral_Test_GUI:
    def __init__(self, window):
        #Initialize the window variable
        self.window = window
        
        #Define variables
        self.qs_counter = 1
        self.score = 0
        self.ans_1 = ""
        self.ans_2 = ""
        self.ans_3 = ""
        self.ans_4 = ""
        self.ans_5 = ""
        
        #Window settings, title and size
        window.title("ORAL TEST")
        self.window.geometry("2000x2000") 
        self.window.resizable(width="true", height="true")
        
        #Divide window in frames
        self.frame_header = tk.Frame(self.window, borderwidth=5, pady=2)
        self.center_frame = tk.Frame(self.window, borderwidth=5, pady=5)
        self.bottom_frame = tk.Frame(self.window, borderwidth=5, pady=5)
        
        #Assign grid to the frames. Grid is a geometry manager that organizes widgets in a table-like structure in the 
        #parent widget. Any location in the grid can be accessed by specifying row and column.
        self.frame_header.grid(row=0, column=0)
        self.center_frame.grid(row=1, column=0)
        self.bottom_frame.grid(row=2, column=0)

        #Label header to be placed in the frame_header
        self.header = tk.Label(self.frame_header, text = "KINDERGARTEN ORAL TEST", bg='orange red', fg='yellow', height='3', width='90', font=("Calibri 24 bold"))
        #position of the header inside the grid of frame_header(0,0)
        self.header.grid(row=0, column=0)

        #Define two additional frames go inside the center_frame
        self.frame_main_1 = tk.Frame(self.center_frame, borderwidth=3, relief='sunken')
        self.frame_main_2 = tk.Frame(self.center_frame, borderwidth=3, relief='sunken')

        #Populate them with the labels referring to the inputs we want from the user
        self.name = tk.Label(self.frame_main_1, text = "Enter your Name : ", font=("Helvetica 12"))
        self.roll_number = tk.Label(self.frame_main_2, text = "Enter your Roll Number :      ", font=("Helvetica 12"))

        #StringVar() allows you to easily track tkinter variables and see if they were read, changed, etc
        self.name_var = tk.StringVar()
        self.roll_number_var = tk.StringVar()

        #Display the labels inside the center frame. The order which we pack the items is important
        self.frame_main_1.pack(fill='x', pady=2)
        self.frame_main_2.pack(fill='x',pady=2)
        self.name.pack(side='left')
        self.roll_number.pack(side='left')
        
        #Create the entries for the user input - name and roll number
        self.name_entry = tk.Entry(self.frame_main_1, textvariable = self.name_var, width=10)
        self.roll_number_entry = tk.Entry(self.frame_main_2, textvariable = self.roll_number_var, width=10)

        #Everytime a key is released, it runs the name_caps and roll_number_validity functions on the respective cells
        self.name_entry.bind("<KeyRelease>", self.name_caps)
        self.roll_number_entry.bind("<KeyRelease>", self.roll_number_validity) 

        #Pack the two frames in the center_frame and then the elements inside them
        self.frame_main_1.pack(fill='x', pady=2)
        self.frame_main_2.pack(fill='x',pady=2)

        #The order which we pack the items is important
        self.name.pack(side='left')
        self.name_entry.pack(side='left', padx=1)
        self.roll_number.pack(side='left')
        self.roll_number_entry.pack(side='left', padx=1)

        #Add and place the 'Done' button which is displayed when we begin the app
        self.button_done = tk.Button(self.bottom_frame, text="Done", bg='RoyalBlue1', state = "disabled",fg='black', relief='raised', width=10, font=('Helvetica 10'))
        self.button_done.grid(column=0, row=0, sticky='w', padx=100, pady=2)
        self.button_done.bind('<Button-1>', self.validate_done_button)

        #Initialize other buttons for our app
        self.button_start = tk.Button(self.bottom_frame)
        self.button_close = tk.Button(self.bottom_frame)
        self.button_restart = tk.Button(self.bottom_frame)
        self.button_report = tk.Button(self.bottom_frame)
        self.button_graphs = tk.Button(self.bottom_frame)
        #button_next = tk.Button(window)#.place(x=1350, y=680)
        self.button_next = tk.Button(self.bottom_frame)
        
        #Load introduction and six header images
        #Introduction image
        self.load = Image.open("intro.png")#255,69,0
        self.render = ImageTk.PhotoImage(self.load)
        self.img = tk.Label(window, image=self.render)
        self.img.configure(image = self.render)
        self.img.image = self.render
        self.img.place(x=75, y=290)
        
        #Header image 1
        load_intro_image_1 = Image.open("intro_1.png")
        load_intro_image_1 = load_intro_image_1.resize((120,120), Image.ANTIALIAS)
        render_1 = ImageTk.PhotoImage(load_intro_image_1)
        img_1 = tk.Label(window, image=render_1)
        img_1.configure(image = render_1)
        img_1.image = render_1
        img_1.place(x=1350, y=7)

        #Header image 2
        load_intro_image_2 = Image.open("intro_2.png")
        load_intro_image_2 = load_intro_image_2.resize((140,120), Image.ANTIALIAS)
        render_2 = ImageTk.PhotoImage(load_intro_image_2)
        img_2 = tk.Label(window, image=render_2)
        img_2.configure(image = render_2)
        img_2.image = render_2
        img_2.place(x=50, y=7)

        #Header image 3
        load_intro_image_3 = Image.open("intro_3.png")
        load_intro_image_3 = load_intro_image_3.resize((140,100), Image.ANTIALIAS)
        render_3 = ImageTk.PhotoImage(load_intro_image_3)
        img_3 = tk.Label(window, image=render_3)
        img_3.configure(image = render_3)
        img_3.image = render_3
        img_3.place(x=240, y=15)

        #Header image 4
        load_intro_image_4 = Image.open("intro_4.png")
        load_intro_image_4 = load_intro_image_4.resize((140,100), Image.ANTIALIAS)
        render_4 = ImageTk.PhotoImage(load_intro_image_4)
        img_4 = tk.Label(window, image=render_4)
        img_4.configure(image = render_4)
        img_4.image = render_4
        img_4.place(x=1160, y=15)

        #Header image 5
        load_intro_image_5 = Image.open("intro_5.png")
        load_intro_image_5 = load_intro_image_5.resize((120,140), Image.ANTIALIAS)
        render_5 = ImageTk.PhotoImage(load_intro_image_5)
        img_5 = tk.Label(window, image=render_5)
        img_5.configure(image = render_5)
        img_5.image = render_5
        img_5.place(x=430, y=7)

        #Header image 6
        load_intro_image_6 = Image.open("intro_6.png")
        load_intro_image_6 = load_intro_image_6.resize((120,140), Image.ANTIALIAS)
        render_6 = ImageTk.PhotoImage(load_intro_image_6)
        img_6 = tk.Label(window, image=render_6)
        img_6.configure(image = render_6)
        img_6.image = render_6
        img_6.place(x=990, y=7)

    def name_caps(self, event):
        """Validates name to contain alphabets only.
        Converts name input from the user to upper case."""
        if not self.name_var.get().isalpha():
            messagebox.showinfo("Title", "Invalid name. Enter alphabets only.")
        else:
            self.name_var.set(self.name_var.get().upper())
            
    def roll_number_validity(self, event):
        """Forces roll number input from user to be less than 4 characters and a digit"""
        if len(self.roll_number_var.get()) > 2 or not self.roll_number_var.get().isdigit():
            messagebox.showinfo("Title", "Invalid roll number. Enter numbers less than 2.")
         
    def validate_done_button(self,event):
        """Check if correct name and roll number is entered before activating 'Done' button and moving to the next screen"""
        name = self.name_var.get() 
        roll_num = self.roll_number_var.get()
        self.button_done.config(state = "normal", command = self.run_app)
        
    def close_app(self):
        """Close window"""
        self.window.destroy()

    def run_app(self):
        """This function is triggered after the name and roll number validation.
        Play the welcome message and enable the 'Start' button to behin test by calling begin_test function on button click.
        Enable the 'Exit' button to end the test."""

        #Disable name and roll number text boxes to avoid changes once submitted. Remove the 'Done' button
        self.name_entry.config(state ='disabled')
        self.roll_number_entry.config(state='disabled')
        self.button_done.grid_remove()

        #Create audio file for the welcome message using text to speech conversion. If file already exists, use it else 
        #create a new one
        file_exists = not os.path.exists("welcome.mp3")
        if file_exists:
            introduction = 'Welcome to the online testing session! You will look at the picture and answer the question in one word. Press the Start button to begin!'
            intro = gTTS(text=introduction, lang="en", slow=False) 
            intro.save("welcome.mp3") 
            mixer.music.load("welcome.mp3")
            mixer.music.play()
        else:
            mixer.music.load("welcome.mp3")
            mixer.music.play()

        #Configure and place the 'Start' button. Link 'begin_test function' to be executed when button is clicked.
        self.button_start.config(text="Start", command=self.begin_test, bg='dark green', fg='black', relief='raised', width=10, font=('Helvetica 10'))
        self.button_start.grid(column=1, row=0, sticky='e', padx=100, pady=2)

        #Configure and place the 'Exit' button. Link 'close_app function' to be executed when button is clicked.
        self.button_close.config(text="Exit", command=self.close_app, bg='dark red', fg='black', relief='raised', width=10, font=('Helvetica 10'))
        self.button_close.grid(column=2, row=0, sticky='e', padx=100, pady=2)
        
    def begin_test(self):    
        """This function is triggered on the click of the 'Start' button and the 'Next' button.
        #Calls the 'answer_validation' function for each question. 
        At the end of questions, displayes the score, enables 'Report', 'Graphs' and 'Restart' buttons and displayes the 
        completed image.
        Creates database and table if it does not exist and insert details of new student and save the dataframe to the local
        disk"""
      
        #Remove the 'Start' button as the test has already started
        self.button_start.grid_remove()

        #Define test questions
        question_1_text = 'Question 1: Count the number of balls.'
        question_2_text = 'Question 2: Fill in the blank'
        question_3_text = 'Question 3: What shape do you see?'
        question_4_text = 'Question 4: What is the name of the circled body part?'
        question_5_text = 'Question 5: What is the colour of the Sun?'

        #Pass correct parameters(question text, name of the question audio, image name, answer and question number) 
        #to the 'answer_validation' function
        if self.qs_counter == 1:
            self.answer_validation(question_1_text, "question_1.mp3", "balls.png", "5", 1)
        elif self.qs_counter == 2:
            self.answer_validation(question_2_text, "question_2.mp3", "eating.png", "eating", 2)
        elif self.qs_counter == 3:
            self.answer_validation(question_3_text, "question_3.mp3", "circle.png", "Circle", 3)
        elif self.qs_counter == 4:
            self.answer_validation(question_4_text, "question_4.mp3", "chin.png", "chin", 4)
        elif self.qs_counter == 5:
            self.answer_validation(question_5_text, "question_5.mp3", "yellow.png", "yellow", 5)
        else:
            #Convert score to string and display in a message box
            score_in_str = "You scored: " + str(self.score)
            score_str = str(self.score) 
            messagebox.showinfo("Score", score_in_str)

            #Remove the 'Next button'
            #button_next.place_forget()
            self.button_next.grid_remove()

            #Configure and place the restart button to execute 'restart_app' function on button click
            self.button_restart.config(text="Restart", command=self.restart_app, bg='blue', fg='black', relief='raised', width=10, font=('Helvetica 10'))
            self.button_restart.grid(column=3, row=0, sticky='e', padx=100, pady=2)

            #Configure and place the report button to execute 'generate_report' function on button click
            self.button_report.config(text="Report", command = self.generate_report, bg='purple2', fg='black', relief='raised', width=10, font=('Helvetica 10'))
            self.button_report.grid(column=4, row=0, sticky='e', padx=100, pady=2)

            #Configure and place the graphs button to execute 'generate_graphs' function on button click
            self.button_graphs.config(text="Graphs", command = self.generate_graphs, bg='magenta2', fg='black', relief='raised', width=10, font=('Helvetica 10'))
            self.button_graphs.grid(column=5, row=0, sticky='e', padx=100, pady=2)

            #Destroy previous image
            self.img.destroy()

            #Load the completed image after the test concludes
            self.load = Image.open("completed.png")#255,69,0
            self.render = ImageTk.PhotoImage(self.load)
            self.img = tk.Label(window, image=self.render)
            self.img.configure(image = self.render)
            self.img.image = self.render
            self.img.place(x=75, y=290)    

            #Path where we want to create the database
            db_filename = "kg_oral_exam.db"
            db_exists = not os.path.exists(db_filename)

            if db_exists:
                connection = None
                #Connect to the database
                connection = sqlite3.connect(db_filename)
                print('Database created.')
                connection.close()
            else:
                print('DB already exists.')

            #Create table
            kg_oral_marks = """ CREATE TABLE IF NOT EXISTS kg_oral_marks (
                                                name text,
                                                roll_num text,
                                                score_str text,
                                                answer_1 text,
                                                answer_2 text,
                                                answer_3 text,
                                                answer_4 text,
                                                answer_5 text); """

            #Create a database connection
            connection = None
            connection = sqlite3.connect(db_filename)

            #Get user input from text box and execute the insert query to insert data in the table
            if connection is not None:
                name = self.name_var.get() 
                roll_num = self.roll_number_var.get()
                c = connection.cursor()
                #c.execute("DROP TABLE world_news;");
                c.execute(kg_oral_marks)
                print('Table created.')
                c.execute('INSERT INTO kg_oral_marks (name, roll_num, score_str, answer_1, answer_2, answer_3, answer_4, answer_5) VALUES (?,?,?,?,?,?,?,?)', (name, roll_num, score_str, self.ans_1, self.ans_2, self.ans_3, self.ans_4, self.ans_5))
                print('Data inserted in the table.')
                connection.commit()
            else:
                print("Error! cannot create the database connection.")
            query_1 = "select * from kg_oral_marks"

            #Read data from the table and save it to a DataFrame
            marks_df = pd.read_sql_query(query_1, connection)
            marks_df.to_csv("kg_marks.csv",index=False)

            #Close connection to the database
            connection.close()
    
    def answer_validation(self, question_text, file_name, image_name, ans, qs_num):
        """This function is called from the 'begin_test' function.
        It reads the question from the question text, displayes the question and waits for answer from the child.
        Once answer is received, it is checked and if it is correct, global variable score is incremented. 
        Answer given by the child for each question is saved for further analysis and displayed in the report.
        'Next' button is configured and placed and executed the 'begin_test' function on button click."""
        #Destroy previous image
        self.img.destroy()

        #Remove the next button temporarily
        self.button_next.grid_remove()

        #Play audio for the question
        file_exists = not os.path.exists(file_name)
        if file_exists:
            q1 = gTTS(text=question_text, lang="en", slow=False) 
            q1.save(file_name) 
            mixer.music.load(file_name)
            mixer.music.play()
        else:
            mixer.music.load(file_name)
            mixer.music.play()

        #Load image for the question
        self.load = Image.open(image_name)
        self.render = ImageTk.PhotoImage(self.load)
        self.img = tk.Label(window, image=self.render)
        self.img.image = self.render
        self.img.place(x=75, y=290)#160,300

        #Initialize speech recognizer
        r = sr.Recognizer()
        var = tk.IntVar()

        #Wait for the answer
        window.after(3550, var.set, 1)
        window.wait_variable(var)

        #Using microphone save the anwer
        with sr.Microphone() as source:
        #r.adjust_for_ambient_noise(source) 
            audio = r.listen(source)#, timeout = 10)

        try:
            #Convert the answer to text from the audio
            answer = r.recognize_google(audio)

            #Check answer
            if answer == ans:
                self.score = self.score + 1

            #Save answers for each question
            if qs_num == 1:
                self.ans_1 = answer
            elif qs_num == 2:
                self.ans_2 = answer
            elif qs_num == 3:
                self.ans_3 = answer
            elif qs_num == 4:
                self.ans_4 = answer
            elif qs_num == 5:
                self.ans_5 = answer

        except:
            no_answer = "No answer received."
            file_exists = not os.path.exists("no_ans.mp3")
            if file_exists:
                no_ans = gTTS(text=no_answer, lang="en", slow=False) 
                no_ans.save("no_ans.mp3") 
                mixer.music.load("no_ans.mp3")
                mixer.music.play()
            else:
                mixer.music.load("no_ans.mp3")
                mixer.music.play()
            window.after(1550, var.set, 1)
            window.wait_variable(var)

        #Increment counter used in 'begin_test' function to move to the next question
        self.qs_counter += 1

        #Play audio for the 'Next' button, position it and configure to again execute 'begin_test' function on button click
        next_button = "Press the next button."
        file_exists = not os.path.exists("next_button.mp3")
        if file_exists:
            next_qs = gTTS(text=next_button, lang="en", slow=False) 
            next_qs.save("next_button.mp3") 
            mixer.music.load("next_button.mp3")
            mixer.music.play()
        else:
            mixer.music.load("next_button.mp3")
            mixer.music.play()
        
        #Call begin_test function again for the next question
        self.button_next.config(text="Next",command=self.begin_test, bg='orange red', fg='black', relief='raised', width=10, font=('Helvetica 10'))
        self.button_next.grid(column=3, row=0, sticky='e', padx=100, pady=2)
        
        
    def generate_report(self):
        """This function is triggered on the click of the 'Report' button.
        Generates report containing student name, roll number, score and the answers to the questions"""
        #Destroy the previous image before displaying the report
        self.img.destroy() 

        #Add one more frame to place and position the report
        more_bottom_frame = tk.Frame(window, borderwidth=15, pady=5)
        more_bottom_frame.place(x=200, y=265, width=1150, height=530)

        #Read the saved csv file from disk
        kg_marks = pd.read_csv(r'''kg_marks.csv''')
        table = kg_marks

        #Create a table frame with heading
        report_frame = tk.LabelFrame(more_bottom_frame, text='Table editor')
        report_frame.pack(fill='x',expand=True)
        report_frame.pack()

        #Create table in that frame and display the marks table
        report_table = Table(report_frame, dataframe=table, showtoolbar=True, showstatusbar=True)
        report_table.show()
    
    def generate_graphs(self):
        """This function is triggered on the click of the 'Graphs' button.
        Generates graphs for detailed analysis on student performance"""
        #Load replacement image to place a white screen before loading the graphs
        self.load = Image.open("replacement.png")
        self.render = ImageTk.PhotoImage(self.load)
        self.img = tk.Label(window, image=self.render)
        self.img.image = self.render
        self.img.place(x=0, y=270)

        #Read the saved csv file from disk
        marks = pd.read_csv(r'''kg_marks.csv''')
        %matplotlib inline

        #Graph 1 - Plot a bar graph to display count of students for each score
        marks_groups = marks.groupby("score_str", as_index = False)["name"].count()
        plt.bar(marks_groups["score_str"], marks_groups["name"], color = (0.6,0.20,0.2,0.8))
        plt.ylabel('Number of students')
        plt.title('Score-wise student count')
        plt.savefig('Score wise student count.png')
        plt.clf()

        #Graph 2 - Plot a pie chart to display percentage of students for each score
        plt.pie(marks_groups["name"],autopct='%1.1f%%', startangle=90)
        plt.axis('equal')
        plt.legend( loc = 'right', labels=marks_groups["score_str"])
        plt.title("Score-wise student percentage")
        plt.savefig('Score-wise student percentage.png')
        plt.clf()

        #Graph 3 - Plot a bar graph displaying the class statistics
        marks["score_str"].describe()[["mean", "min", "max", "25%", "50%", "75%"]].plot(style='.-', title = "Marks statistics", color = (0.6,0.20,0.2,0.8))
        plt.savefig('Score statistics')
        plt.clf()

        #Graph 4 - Plot a bar graph displaying list of students and their scores who need re-teaching
        low_marks = marks[marks["score_str"] <=2]
        plt.bar(low_marks["name"], low_marks["score_str"], color = (0.6,0.20,0.2,0.8))
        plt.xticks(rotation=45)
        plt.ylabel('Score')
        plt.title('Re-teach student list')
        plt.savefig('Reteach student list.png')
        plt.clf()

        #Load and place graph 1 on the screen
        load_graph_1 = Image.open("Score wise student count.png")
        load_graph_1 = load_graph_1.resize((320,220), Image.ANTIALIAS)
        render_graph_1 = ImageTk.PhotoImage(load_graph_1)
        img_graph_1 = tk.Label(window, image=render_graph_1)
        img_graph_1.image = render_graph_1
        img_graph_1.place(x=200, y=285)

        #Load and place graph 2 on the screen
        load_graph_2 = Image.open("Score-wise student percentage.png")
        load_graph_2 = load_graph_2.resize((320,220), Image.ANTIALIAS)
        render_graph_2 = ImageTk.PhotoImage(load_graph_2)
        img_graph_2 = tk.Label(window, image=render_graph_2)
        img_graph_2.image = render_graph_2
        img_graph_2.place(x=600, y=285)

        #Load and place graph 3 on the screen
        load_graph_3 = Image.open("Score statistics.png")
        load_graph_3 = load_graph_3.resize((320,220), Image.ANTIALIAS)
        render_graph_3 = ImageTk.PhotoImage(load_graph_3)
        img_graph_3 = tk.Label(window, image=render_graph_3)
        img_graph_3.image = render_graph_3
        img_graph_3.place(x=1000, y=285)

        #Load and place graph 4 on the screen
        load_graph_4 = Image.open("Reteach student list.png")
        load_graph_4 = load_graph_4.resize((320,220), Image.ANTIALIAS)
        render_graph_4 = ImageTk.PhotoImage(load_graph_4)
        img_graph_4 = tk.Label(window, image=render_graph_4)
        img_graph_4.image = render_graph_4
        img_graph_4.place(x=600, y=520)
        
        
    def restart_app(self):
        """This function is triggered on the click of the 'Restart' button.
        Function to restart the oral exam for the next student. Reset variables, buttons and images to the original page."""
        #Restore the text boxes to normal state to accept input from user
        self.name_entry.config(state ='normal')
        self.roll_number_entry.config(state='normal')

        #Delete previous student's text information
        self.name_entry.delete(0, 'end')
        self.roll_number_entry.delete(0, 'end')

        #Remove buttons not required
        self.button_start.grid_remove()
        self.button_close.grid_remove()
        self.button_restart.grid_remove()
        self.button_report.grid_remove()
        self.button_graphs.grid_remove()
        #button_close_report.place_forget()

        #Load the introduction image again
        self.load = Image.open("intro.png")
        self.render = ImageTk.PhotoImage(self.load)
        self.img = tk.Label(window, image=self.render)
        self.img.image = self.render
        self.img.place(x=75, y=290)

        #Rest global variables
        self.qs_counter = 1
        self.score = 0
        self.ans_1 = ""
        self.ans_2 = ""
        self.ans_3 = ""
        self.ans_4 = ""
        self.ans_5 = ""

        #Position the 'Done' button and link it to validate_done_button function
        self.button_done.grid(column=0, row=0, sticky='w', padx=100, pady=2)
        self.button_done.config(state = "disabled")
        self.button_done.bind('<Button-1>', self.validate_done_button)

#Root window of the application
window = tk.Tk()

#Initialize the mixer module for audio messages
mixer.init()

my_gui = Oral_Test_GUI(window)

"""This method will loop forever, waiting for events from the user, until the user exits the program – either by closing 
the window, or by terminating the program with a keyboard interrupt in the console."""
window.mainloop()